# Detekcja krawędzi

## Cel ćwiczenia

- Zapoznanie z metodami detekcji krawędzi:
    - Sobel, Prewitt, Roberts - przypomnienie,
    - Laplasjan z Gaussa (LoG – ang. Laplacian of Gaussian),
    - Canny.

Detekcja krawędzi przez wiele lat była podstawą algorytmów segmentacji.
Krawędzie wykrywane są najczęściej z wykorzystaniem pierwszej (gradient) i drugiej (Laplasjan) pochodnej przestrzennej.
Wykorzystanie obu metod zaprezentowane zostało w ćwiczeniu *Przetwarzanie wstępne. Filtracja kontekstowa*.

W niniejszym ćwiczeniu poznane detektory krawędzi zostaną porównane z bardziej zaawansowanymi: Laplasjan z funkcji Gaussa (LoG), Zero Crossing i Canny.

## Laplasjan z Gaussa (LoG)

Funkcja Gaussa:<br>
\begin{equation}
h(r) = e^{\frac{-r^2}{2 \sigma^2}}
\end{equation}<br>
gdzie:
- $r^2 = x^2 + y^2$
- $\sigma$ to odchylenie standardowe.

Działanie filtracji Gaussowskiej zostało przedstawione w ćwiczeniu "Przetwarzanie wstępne". W jej wyniku następuje rozmazanie obrazu.
Laplasjan tej funkcji dany jest wzorem:

\begin{equation}
\nabla^2 h(r) = \frac{r^2 - 2\sigma^2}{\sigma^4} e^{-\frac{r^2}{2\sigma^2}}
\end{equation}

Funkcję (z oczywistych powodów) nazywamy Laplasjan z Gaussa (LoG).
Ponieważ druga pochodna jest operacją liniową, konwolucja obrazu z $\nabla^2 h(r)$ daje taki sam efekt jak zastosowanie filtracji Gaussa na obrazie, a następnie obliczenie Laplasjanu z wyniku.
Lokalizacja krawędzi polega na znalezieniu miejsca, gdzie po filtracji LoG następuje zmiana znaku.

1. Wczytaj obraz *house.png*.
2. Wykonaj rozmycie Gaussowskie obrazu wejściowego.
W tym celu wykorzysaj funkcję `cv2.GaussianBlur(img, kSize, sigma)`.
Pierwszy argument jest obrazem wejśćiowym.
Drugi jest rozmiarem filtru (podanym w nawiasach okrągłych, np. *(3, 3)*).
Trzecim argumentem jest odchylenie standardowe. Wartość jest dobrana automatycznie, jeśli zosanie podana wartość `0` (będą równe rozmiarowi).
3. Oblicz laplasjan obrazu rozmytego.
W tym celu wykorzysaj funkcję `cv2.Laplacian(img, ddepth)`.
Pierszym argumentem jest obraz wejściowy.
Drugim argumentem jest typ danych wejściowych. Użyj `cv2.CV_32F`.
4. Wyznacz miejsca zmiany znaku.
Zaimplementuj funkcję `crossing(LoG, thr)`:
    - Najpierw stwórz tablicę, do której zostanie zapisany wynik.
    Jej rozmiar jest taki sam jak przetwarzanego obrazu.
    - Następnie wykonaj pętle po obrazie (bez ramki jednopikselowej).
    W każdej iteracji stwórz otoczenie o rozmiarze $3 \times 3$.
    Dla otoczenia oblicz wartość maksymalną i minimalną.
    - Jeśli wartości te mają przeciwne znaki, to do danego miejsca tablicy przypisz wartość:
        - jeśli piksel wejściowy > 0, to dodaj do niego wartość bezwzględną minimum.
        - jeśli piksel wejściowy < 0, to do jego wartości bezwzględnej dodaj maksimum.
    - Zmień zakres wykonanej tablicy do $<0, 255>$.
    - Wykonaj probowanie tablicy. Próg jest argumentem wejściowym.
    - Przeskaluj dane binarne do wartości `[0, 255]`.
    - Wykonaj konwersję do typu *uint8*.
    - Wykonaj rozmycie medianowe wyniku.
    Wykorzystaj funkcję `cv2.medianBlur(img, kSize)`.
    Pierwszym argumentem jest obraz wejśćiowy, a drugim rozmiar filtra.
    - Zwróć wyznaczoną tablicę.
5. Wyświetl obraz wynikowy.
6. Dobierz parametry (rozmiar filtru Gaussa, odchylenie standardowe, próg binaryzacji) tak, by widoczne były kontury domu, ale nie dachówki.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os

if not os.path.exists("dom.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/09_Canny/dom.png --no-check-certificate

I_Dom = cv2.imread('dom.png')
I_Dom = cv2.cvtColor(I_Dom, cv2.COLOR_BGR2GRAY)

I_Dom_gb = cv2.GaussianBlur(I_Dom, (3,3), 0)
I_Dom_lap = cv2.Laplacian(I_Dom_gb, cv2.CV_32F)

def LoG(Img, msize,thr,sigma=0):
    I_gb = cv2.GaussianBlur(Img, (msize,msize), sigma)
    I_lap = cv2.Laplacian(I_gb, cv2.CV_32F)
    return crossing(I_lap,thr, msize)

def find_min_max(surr, msize):
    x = msize//2
    val_mid = surr[x,x]
    val_max = surr[0,0]
    val_min = surr[0,0]
    for i in range(msize):
        for j in range(msize):
            if i != x and j != x:
                if surr[i,j] > val_max:
                    val_max = surr[i,j]
                if surr[i,j] < val_min:
                    val_min = surr[i,j]
    if (val_min < 0 and val_max <0) or (val_min>0 and val_max >0):
        return 0
    if val_mid >= 0: return val_mid + abs(val_min)
    else: return abs(val_mid) + val_max

def crossing(LoG, thr, msize):
    x = msize//2
    res = np.zeros(LoG.shape)
    for i in range(x,LoG.shape[0]-x):
        for j in range(x,LoG.shape[1]-x):
            surr = np.zeros((msize,msize))
            for z in range(-x,x+1):
                for y in range(-x,x+1):
                    surr[z+1,y+1] = LoG[i+z,j+y]
                    res[i,j] = find_min_max(surr, msize)
    
    new_res = res / res.max() * 255
    new_res_thr = ((new_res > thr)*255).astype('uint8')
    result = cv2.medianBlur(new_res_thr, msize)
    return result

Input = LoG(I_Dom, 5, 90, 0.5)

plt.imshow(Input, cmap="gray")
plt.xticks([]), plt.yticks([])
plt.show()

## Algorytm Canny'ego

> Algorytm Canny'ego to często wykorzystywana metoda detekcji krawędzi.
> Zaproponowana została w~1986r. przez Johna F. Cannego.
> Przy jego projektowaniu założono trzy cele:
> - niska liczba błędów - algorytm powinien znajdywać wszystkie krawędzie oraz generować jak najmniej fałszywych detekcji,
> - punkty krawędziowe powinny być poprawnie lokalizowane - wykryte punkty powinny być jak najbardziej zbliżone do rzeczywistych,
> - krawędzie o szerokości 1 piksela - algorytm powinien zwrócić jeden punkt dla każdej rzeczywistej krawędzi.

Zaimplementuj algorytm detekcji krawędziCanny'ego:
1. W pierwszym kroku obraz przefiltruj dwuwymiarowym filtrem Gaussa.
2. Następnie oblicz gradient pionowy i poziomy ($g_x $ i $g_y$).
Jedną z metod jest filtracja Sobela.
3. Dalej oblicz amplitudę:
$M(x,y)  = \sqrt{g_x^2+g_y^2}$ oraz kąt:
$\alpha(x,y) = arctan(\frac{g_y}{g_x})$.
Do obliczenia kąta wykorzystaj funkcję `np.arctan2(x1, x2)`.
Wynik jest w radianach.
4. W kolejnym etapie wykonaj kwantyzację kątów gradientu.
Kąty od $-180^\circ$ do $180^\circ$ można podzielić na 8 przedziałów:
[$-22.5^\circ, 22.5^\circ$], [$22.5^\circ, 67.5^\circ$],
[$67.5^\circ, 112.5^\circ$], [$112.5^\circ, 157.5^\circ$],
[$157.5^\circ, -157.5^\circ$], [$-157.5^\circ, -112.5^\circ$],
[$-112.5^\circ, -67.5^\circ$], [$-67.5^\circ, -22.5^\circ$].
Przy czym należy rozpatrywać tylko 4 kierunki:
    - pionowy ($d_1$),
    - poziomy ($d_2$),
    - skośny lewy ($d_3$),
    - skośny prawy ($d_4$).
5. Dalej przeprowadź eliminację pikseli, które nie mają wartości maksymalnej (ang. *nonmaximal suppresion*).
Celem tej operacji jest redukcja szerokości krawędzi do rozmiaru 1 piksela.
Algorytm przebiega następująco.
W rozpatrywanym otoczeniu o rozmiarze $3 \times 3$:
    - określ do którego przedziału należy kierunek gradientu piksela centralnego ($d_1, d_2, d_3, d_4$).
    - przeanalizuje sąsiadów leżących na tym kierunku.
Jeśli choć jeden z nich ma amplitudę większą niż piksel centralny, to należy uznać, że nie jest lokalnym maksimum i do wyniku przypisać $g_N(x,y) = 0$.
W przeciwnym przypadku $g_N(x,y) = M(x,y)$.
Przez $g_N$ rozumiemy obraz detekcji lokalnych maksimów.
Zaimplementuj funkcję `nonmax`.
Pierwszym argementem jest macierz kierunków (po kwantyzacji).
Drugim argumentem jest macierz amplitudy.
6. Ostatnią operacją jest binaryzacja obrazu $g_N$.
Stosuje się tutaj tzw. binaryzację z histerezą.
Wykorzystuje się w niej dwa progi: $T_L$ i $T_H$, przy czym $T_L < T_H$.
Canny zaproponował, aby stosunek progu wyższego do niższego był jak 3 lub 2 do 1.
Rezultaty binaryzacji można opisać jako:<br>
$g_{NH}(x,y) = g_N(x,y) \geq TH $<br>
$g_{NL}(x,y) = TH > g_N(x,y) \geq TL $<br>
Można powiedzieć, że na obrazie $g_{NH}$ są "pewne" krawędzie.
Natomiast na $g_{NL}$ "potencjalne".
Często krawędzie "pewne" nie są ciągłe.
Dlatego wykorzystuje się obraz $g_{NL}$ w następującej procedurze:
    - Stwórz stos zawierający wszystkie piksele zaznaczone na obrazie $g_{NH}$.
W tym celu wykorzystaj listę współrzędnych `[row, col]`.
Do pobrania elementu z początku służy metoda `list.pop()`.
Do dodania elementu na koniec listy służy metoda `list.append(new)`.
    - Stwórz obraz, który będzie zawierał informację czy dany piksel został już odwiedzony.
    - Stwórz obraz, któy zawierać będzie wynikowe krawędzie.
Jej rozmiar jest równy rozmiarowi obrazu.
    - Wykonaj pętlę, która będzie pobierać elementy z listy, dopóki ta nie będzie pusta.
W tym celu najlepiej sprawdzi się pętla `while`.
    - W każdej iteracji pobierz element ze stosu.
    - Sprawdź czy dany element został już odwiedzony.
    - Jeśli nie został, to:
        - Oznacz go jako odwiedzony,
        - Oznacz piksel jako krawędź w wyniku,
        - Sprawdź otoczenie piksela w obrazie $g_{NL}$,
        - Dodaj do stosu współrzędne otoczenia, które zawierają krawędź (potencjalną).
        Można to wykoanać np. pętlą po stworzonym otoczeniu.
7. Wyświetl obraz oryginalny, obraz $g_{NH}$ oraz obraz wynikowy.

Pomocnicze obrazy $g_{NH}$ i $g_{NL}$ zostały wprowadzone dla uproszczenia opisu.
Algorytm można zaimplementować wbardziej "zwarty" sposób.

Na podstawie powyższego opisu zaimplementuj algorytm Cannego.

In [ ]:
def angle_convert(angle):
    x,y = angle.shape
    angle2 = np.zeros_like(angle)
    for i in range(x):
        for j in range(y):
            a = angle[i,j]
            if abs(a) > 180:
                angle2[i,j] = abs(a - 180)
            else:
                angle2[i,j] = abs(a) 
    return angle2

def Canny(img, t_l, t_h, msize=3, sigma=1):
    img = cv2.GaussianBlur(img, (msize, msize), sigma)
    sobelx = cv2.Sobel(np.float32(img), cv2.CV_64F, 1, 0) 
    sobely = cv2.Sobel(np.float32(img), cv2.CV_64F, 0, 1)
    
    mag, ang = cv2.cartToPolar(sobelx, sobely, angleInDegrees = True)
    ang = angle_convert(ang)
    
    non_max = nonmax(ang, mag)
    
    max_mag = np.max(mag)
    #t_l = int(max_mag * 0.2)
    #t_h = int(max_mag * 0.4)
    weak = 150
    strong = 255
    
    tresh, qnh = threshold(non_max, t_l, t_h, weak, strong)
    res = hist(tresh, weak, strong)
    
    fig, axs = plt.subplots(1,3, figsize=(15,15))

    axs[0].set_title("Oryginał")
    axs[0].imshow(img, 'gray')
    axs[0].axis('off')

    axs[1].set_title("QNH")
    axs[1].imshow(qnh, 'gray')
    axs[1].axis('off')

    axs[2].set_title("Wynik")
    axs[2].imshow(res, 'gray')
    axs[2].axis('off')

    plt.show()
    
def nonmax(ang, mag):
    x,y = mag.shape
    res = np.zeros((x,y))
    for i in range(1,x-1):
        for j in range(1,y-1):
            a = 255
            b = 255
            current_angle = ang[i,j]
            if (0 <= current_angle < 22.5) and (157.5 <=  current_angle <= 180):
                a = mag[i - 1, j]
                b = mag[i + 1, j]
            elif 22.5 <= current_angle < 67.5: 
                a = mag[i - 1, j - 1]
                b = mag[i + 1, j + 1]
            elif 67.5 <= current_angle < 112.5: 
                a = mag[i, j - 1]
                b = mag[i, j + 1]
            elif 112.5 <= current_angle < 157.5:
                a = mag[i - 1, j + 1]
                b = mag[i + 1, j - 1]
            
            if mag[i, j] < a or mag[i,j] < b:
                res[i, j] = 0
            else:
                res[i, j] = mag[i,j]
    return res

def threshold(img, low, high, weak, strong):
    res = np.zeros(img.shape)
 
    strong_r, strong_c = np.where(img > high)
    weak_r, weak_c = np.where((img >= low) & (img <= high))
 
    res[strong_r, strong_c] = strong
    qnh = res
    res[weak_r, weak_c] = weak
 
    return res, qnh

def hist(img, weak, strong):
    x, y = img.shape
    for i in range(1, x-1):
        for j in range(1, y-1):
            if (img[i,j] == weak):
                for a in range(i-1,i+2):
                    for b in range(j-1,j+2):
                        if(img[a,b] == strong):
                            img[i,j] == strong
                            break
                        else:
                            img[i,j] = 0
    return img
    
Canny(I_Dom, 100, 200)

## Algorytm Canny'ego - OpenCV

1. Wykonaj dektekcję krawędzi metodą Canny'ego wykorzystując funkcję `cv2.Canny`.
    - Pierwszym argumentem funkcji jest obraz wejściowy.
    - Drugim argumentem jest mniejszy próg.
    - Trzecim argumentem jest większy próg.
    - Czwarty argument to tablica, do której wpisany zostanie wynik.
    Można zwrócić go przez wartość i podać wartość `None`.
    - Piąty argument to rozmiar operatora Sobela (w naszym przypadku 3).
    - Szósty argument to rodzaj używanej normy.
    0 oznacza normę $L_1$, 1 oznacza normę $L_2$. Użyj $L_2$.
2. Wynik wyświetl i porównaj z własną implementacją.

In [ ]:
output = cv2.Canny(I_Dom, 200, 400, None, 3, 2)

plt.imshow(output, cmap="gray")
plt.xticks([]), plt.yticks([])
plt.show()